In [1]:
# -*- coding:utf-8 -*-
import numpy as np
import networkx as nx
from math import sqrt
import time
from utils import myadj2edg

功能：单个根据有序features构建pearson系数OTU关联网络  
输入：单个features文件名，如dpwk.txt  
输出：单个graph文件名，如dpwk.adjlist

In [2]:
def get_pearson_network( in_fname, out_fname, threshhold ):
    
    print('in:{},out:{},threshhold:{} processing...'.format(in_fname, out_fname, threshhold))
    X = np.loadtxt( in_fname, dtype=float)
    m, n = len(X), len(X[0])
    X_mean = np.mean(X, axis=1)
    # sqrt(sum((x-x_avg)^2))
    X_ssfm2 = np.zeros(m) 
    X_first_moment = []
    for i in range(m):
        curr_arr = np.zeros(n)          # x_j - x_avg
        sum_first_moment_2 = 0
        for j in range(n):
            curr_arr[j] = X[i][j] - X_mean[i]
            sum_first_moment_2 += curr_arr[j] * curr_arr[j]
        X_ssfm2[i] = sqrt(sum_first_moment_2)
        X_first_moment.append( curr_arr )
    print('first_moment over')
    
    # 根据一阶矩计算相关系数,构建网络 --只计算上三角矩阵
    G = nx.Graph()                                  
    G.add_nodes_from(np.arange(m))   
    neg_threshhold = -1*threshhold        
    current_edges = 0
    for i in range(m):
        for j in range(i+1,m):                      
            sum = 0
            for k in range(n):
                sum += X_first_moment[i][k] * X_first_moment[j][k]
            sum /= (X_ssfm2[i] * X_ssfm2[j]) 
            if sum > threshhold or sum < neg_threshhold :
                G.add_edge(i,j)
                current_edges += 1
        print('\rnode i:{}, current_edges: {}'.format(i, current_edges),end='')                     
        
    nx.write_adjlist(G, out_fname)           
    print('\nnetwork saved')

功能：宏观控制OTU关联网络构建  
参数：输入、输出文件名、阈值列表

In [3]:
names = ['dpwk', 'line', 'lle', 'n2v']
threshholds = [0.5, 0.7, 0.5, 0.8]
in_fnames = [ './features/'+name+'.txt' for name in names ]
out_fnames = [ './graphs_new/'+name+'.adjlist' for name in names ]

i = 0
print('begin time:{}'.format(time.asctime(time.localtime(time.time()))))    
# get_pearson_network(in_fnames[i], out_fnames[i], threshholds[i] )
print('end time:{}'.format(time.asctime(time.localtime(time.time()))))    

begin time:Sun Apr 25 17:05:44 2021
end time:Sun Apr 25 17:05:44 2021


功能：adjlist文件转换为edgelist文件  
输入：dpwk.adjlist  
输出：dpwk.edgelist, 如 0 1 \n0 6  

In [4]:
# myadj2edg()

./graphs/dpwk.adjlist reading...
./graphs/dpwk.edgelist writing...
./graphs/line.adjlist reading...
./graphs/line.edgelist writing...
./graphs/lle.adjlist reading...
./graphs/lle.edgelist writing...
./graphs/n2v.adjlist reading...
./graphs/n2v.edgelist writing...
